# Using Clustering to Reduce Log Error in Zillow Dataset

## Project Plan

### Goals

### Hypothesis

### Components

In [2]:
import acquire
import prep
import split_scale

## Acquire and Prep

### Pull SQL data and filter data frame

Using the `get_sql_zillow` function from the `acquire` module, pull all the columns from the Zillow dataset from SQL following the conditions below:
 - only include properties that have recorded transactions in 2017
 - use a left join to merge all tables of descriptions on their associated id property
 - exclude properties that do not have latitude and longitude values (location is important in our analysis later so we wante to make sure we are including observations with known locations.)

In [3]:
# get_sql_zillow code here
# df = acquire.get_sql_zillow()

In [4]:
import pandas as pd

# read sql data from csv to minimize lag
df = pd.read_csv(r'/Users/mists/codeup-data-science/zillow-cluster/query_result.csv')
# df = pd.read_csv(r'/Users/cris/codeup-data-science/zillow-cluster/query_result.csv')

Using the `wrangle_zillow` function, perform initial cleaning of the original dataframe specifically:
 - Drop redundant ID columns from Left Joins (typeconstructionid, storytypeid, etc.)
 - Remove duplicates from multiple transactions by maintaining latest transaction date and dropping previous records
 - Include property land use type "Single Family Residential" only, and drop the rest of the observations
 - Drop properties that have unit counts of 2 (27 rows) and 3 (1 row) to remove ambiguity in the definition of "single-unit" or "single-family" houses, especially since the total number of these observations are relatively minimal.
 - Keep transactions that are recorded in 2017 (there was 1 row that has 2018 as transaction year)

In [5]:
# wrangle_zillow code here
df = acquire.wrangle_zillow(df)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52292 entries, 77578 to 0
Data columns (total 61 columns):
parcelid                        52292 non-null int64
basementsqft                    47 non-null float64
bathroomcnt                     52292 non-null float64
bedroomcnt                      52292 non-null int64
buildingqualitytypeid           33632 non-null float64
calculatedbathnbr               52158 non-null float64
decktypeid                      388 non-null float64
finishedfloor1squarefeet        4368 non-null float64
calculatedfinishedsquarefeet    52211 non-null float64
finishedsquarefeet12            52047 non-null float64
finishedsquarefeet13            0 non-null float64
finishedsquarefeet15            0 non-null float64
finishedsquarefeet50            4368 non-null float64
finishedsquarefeet6             164 non-null float64
fips                            52292 non-null int64
fireplacecnt                    7230 non-null float64
fullbathcnt                     521

### Handling missing values

**Drop Unsalvagable Columns and Rows**

`handle_missing_values` function from the `prep` module drops columns that are 90% empty, reducing our columns from 60 to 27. Then, remove observations that are 40% empty. The latter removed no observations.  
  
In this stage, our working data has 52292 rows with 27 columns.

In [7]:
df = prep.handle_missing_values(df,.90,.40)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52292 entries, 77578 to 0
Data columns (total 28 columns):
parcelid                        52292 non-null int64
bathroomcnt                     52292 non-null float64
bedroomcnt                      52292 non-null int64
calculatedbathnbr               52158 non-null float64
calculatedfinishedsquarefeet    52211 non-null float64
finishedsquarefeet12            52047 non-null float64
fips                            52292 non-null int64
fullbathcnt                     52158 non-null float64
latitude                        52292 non-null int64
longitude                       52292 non-null int64
lotsizesquarefeet               51930 non-null float64
propertycountylandusecode       52292 non-null object
rawcensustractandblock          52292 non-null float64
regionidcity                    51256 non-null float64
regionidcounty                  52292 non-null int64
regionidzip                     52266 non-null float64
roomcnt                 

**Drop Non-value-adding Columns**

*dropping parcelid*

`bathroomcnt` reflects the same information as the `calculatedbathnbr`, which is the number of bathrooms in a property including half bathrooms (.5's). The	`fullbathcnt` column only includes full bathrooms and discounts half baths. So, we are discarding redundant `calculatedbathnbr` and `fullbathcnt` and keep `bathroomcnt` to provide information on the properties' bathrooms.

`calculatedfinishedsquarefeet` and `finishedsquarefeet12` hold practically the same information. `calculatedfinishedsquarefeet` has less nulls (81 rows) than `finishedsquarefeet12` (245 rows), so we are dropping the `finishedsquarefeet12`.

`propertycountylandusecode` are codes used in the industry to specify the land use. For example, a Single Family Residential property land use type may be Single Family Class II (0102), Vacant Residential (0000), Vacant Lake View (0035), etc. We don't need these further information. All we need to know is that the properties that we are looking at are under the umbrella of "Single Family Residential."

`rawcensustractandblock` and `censustractandblock` contain census information that we are not concerned about in this project.

`roomcnt` has 37,588 properties with 0 rooms, so we are dropping this column.

`fips` and `regionidcounty` have cotain the same information. We will keep the `fips` column.

`regionidzip` and `regionidcity` are location-based columns which do not give much added information on the properties' location given that we have `latitude` and `longitude`. These columns are dropped.

All wrangled observation have "2016" as values in the `assessmentyear` which is not adding value to the analysis.

`propertylandusedesc` is dropped because it only displays "Single Family Residential" which was helpful in the filtering phase but not useful in the analysis.

In [9]:
df = prep.clean_columns(df)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52292 entries, 77578 to 0
Data columns (total 15 columns):
bathroomcnt                     52292 non-null float64
bedroomcnt                      52292 non-null int64
calculatedfinishedsquarefeet    52211 non-null float64
fips                            52292 non-null int64
latitude                        52292 non-null int64
longitude                       52292 non-null int64
lotsizesquarefeet               51930 non-null float64
yearbuilt                       52178 non-null float64
structuretaxvaluedollarcnt      52213 non-null float64
taxvaluedollarcnt               52291 non-null float64
landtaxvaluedollarcnt           52291 non-null float64
taxamount                       52288 non-null float64
id.1                            52292 non-null int64
logerror                        52292 non-null float64
transactiondate                 52292 non-null object
dtypes: float64(9), int64(5), object(1)
memory usage: 6.4+ MB


**Handle Nulls for Relevant Columns**

In [11]:
# show null values
df.isnull().sum()

bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet     81
fips                              0
latitude                          0
longitude                         0
lotsizesquarefeet               362
yearbuilt                       114
structuretaxvaluedollarcnt       79
taxvaluedollarcnt                 1
landtaxvaluedollarcnt             1
taxamount                         4
id.1                              0
logerror                          0
transactiondate                   0
dtype: int64

_Calculated Finished Square Feet, Structure Tax Value Dollar Count_ -  Observations that have missing values in either `calculatedfinishedsquarefeet` (71 rows) or  `structuretaxvaluedollarcnt` (69 rows), both (10 rows) are dropped, removing a total of 150 observations.

_Tax Value Dollar Count, Land Tax Value Dollar Count_ - There are a handful of observations with missing values in the columns `taxvaluedollarcnt` (1 row), `landtaxvaluedollarcnt` (1 row), `taxamount` (4 rows).

_Year Built_ - After all the above removals, `yearbuilt` have a total of 38 missing values which are dropped because of the manageable size.

In [12]:
df = prep.drop_minimal_nulls(df)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52101 entries, 77578 to 0
Data columns (total 15 columns):
bathroomcnt                     52101 non-null float64
bedroomcnt                      52101 non-null int64
calculatedfinishedsquarefeet    52101 non-null float64
fips                            52101 non-null int64
latitude                        52101 non-null int64
longitude                       52101 non-null int64
lotsizesquarefeet               51761 non-null float64
yearbuilt                       52101 non-null float64
structuretaxvaluedollarcnt      52101 non-null float64
taxvaluedollarcnt               52101 non-null float64
landtaxvaluedollarcnt           52101 non-null float64
taxamount                       52101 non-null float64
id.1                            52101 non-null int64
logerror                        52101 non-null float64
transactiondate                 52101 non-null object
dtypes: float64(9), int64(5), object(1)
memory usage: 6.4+ MB


Renamed and reordered columns

In [14]:
df = prep.pretty_cols(df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52101 entries, 77578 to 0
Data columns (total 14 columns):
countyid           52101 non-null int64
latitude           52101 non-null int64
longitude          52101 non-null int64
yearbuilt          52101 non-null float64
bathroomcnt        52101 non-null float64
bedroomcnt         52101 non-null int64
house_area         52101 non-null float64
house_value        52101 non-null float64
land_value         52101 non-null float64
whole_area         51761 non-null float64
whole_value        52101 non-null float64
taxamount          52101 non-null float64
logerror           52101 non-null float64
transactiondate    52101 non-null object
dtypes: float64(9), int64(4), object(1)
memory usage: 6.0+ MB


### Derived Features

In [16]:
df = prep.cal_taxrate(df)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52101 entries, 77578 to 0
Data columns (total 14 columns):
countyid           52101 non-null int64
latitude           52101 non-null int64
longitude          52101 non-null int64
yearbuilt          52101 non-null float64
bathroomcnt        52101 non-null float64
bedroomcnt         52101 non-null int64
house_area         52101 non-null float64
house_value        52101 non-null float64
land_value         52101 non-null float64
whole_area         51761 non-null float64
whole_value        52101 non-null float64
logerror           52101 non-null float64
transactiondate    52101 non-null object
taxrate            52101 non-null float64
dtypes: float64(9), int64(4), object(1)
memory usage: 6.0+ MB


In [18]:
# df.transactiondate = pd.to_datetime(df.transactiondate)
df.countyid = df.countyid.astype("category")

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52101 entries, 77578 to 0
Data columns (total 14 columns):
countyid           52101 non-null category
latitude           52101 non-null int64
longitude          52101 non-null int64
yearbuilt          52101 non-null float64
bathroomcnt        52101 non-null float64
bedroomcnt         52101 non-null int64
house_area         52101 non-null float64
house_value        52101 non-null float64
land_value         52101 non-null float64
whole_area         51761 non-null float64
whole_value        52101 non-null float64
logerror           52101 non-null float64
transactiondate    52101 non-null datetime64[ns]
taxrate            52101 non-null float64
dtypes: category(1), datetime64[ns](1), float64(9), int64(3)
memory usage: 5.6 MB


## Split

In [20]:
train, test = split_scale.split_my_data(df, .80)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41680 entries, 28235 to 54472
Data columns (total 14 columns):
countyid           41680 non-null category
latitude           41680 non-null int64
longitude          41680 non-null int64
yearbuilt          41680 non-null float64
bathroomcnt        41680 non-null float64
bedroomcnt         41680 non-null int64
house_area         41680 non-null float64
house_value        41680 non-null float64
land_value         41680 non-null float64
whole_area         41405 non-null float64
whole_value        41680 non-null float64
logerror           41680 non-null float64
transactiondate    41680 non-null datetime64[ns]
taxrate            41680 non-null float64
dtypes: category(1), datetime64[ns](1), float64(9), int64(3)
memory usage: 4.5 MB


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10421 entries, 37917 to 44340
Data columns (total 14 columns):
countyid           10421 non-null category
latitude           10421 non-null int64
longitude          10421 non-null int64
yearbuilt          10421 non-null float64
bathroomcnt        10421 non-null float64
bedroomcnt         10421 non-null int64
house_area         10421 non-null float64
house_value        10421 non-null float64
land_value         10421 non-null float64
whole_area         10356 non-null float64
whole_value        10421 non-null float64
logerror           10421 non-null float64
transactiondate    10421 non-null datetime64[ns]
taxrate            10421 non-null float64
dtypes: category(1), datetime64[ns](1), float64(9), int64(3)
memory usage: 1.1 MB


### Must split into train and test first

_Lot Size Sqft_ - The process for imputing `lotsizesquarefeet` is the least straightforward in terms of handling missing values. Lot size sqft is the sum of the area of the land without structure (land or dirt) and the finished space (structure or house). The column has 340 missing values.

Below is the process for filling the missing values on `lotsizesquarefeet`:

1. Derive the total tax dollar value of the lot by adding the tax value of the land and the tax value of the structure

  $ value_{lot} = value_{land} + value_{structure} $
  
  
2. Get the proportion of the lot area and the lot tax value

  $ proportion = \frac {area_{lot}}{value_{lot}} $
  

3. There were unrealistic proportions that seem to come from properties that have a big lot area but low tax value. At 75 percentile, we see a proportion of 0.041 and a mean of 0.048. Because of this, we are confident to take the mean of all proportions less than 1 (proportion < 1). The mean will inform us a generalized value for the lot square footage, i.e., $area_{lot}$, given the total lot value, i.e., $value_{lot}$.

  
4. Impute the nulls in the `lotsizesquarefeet` aka $value_{lot}$ using the derived formula:

  $ area_{lot} = value_{lot} * \mu_{proportion} $

 

In [23]:
train, test = prep.impute_lotsize_nulls(train, test)

In [24]:
train.isnull().sum()

countyid           0
latitude           0
longitude          0
yearbuilt          0
bathroomcnt        0
bedroomcnt         0
house_area         0
house_value        0
land_value         0
whole_area         0
whole_value        0
logerror           0
transactiondate    0
taxrate            0
dtype: int64

In [25]:
test.isnull().sum()

countyid           0
latitude           0
longitude          0
yearbuilt          0
bathroomcnt        0
bedroomcnt         0
house_area         0
house_value        0
land_value         0
whole_area         0
whole_value        0
logerror           0
transactiondate    0
taxrate            0
dtype: int64

Created a land area column from lot size and calculated house size

In [26]:
train = prep.cal_land_area(train)
test = prep.cal_land_area(test)

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41680 entries, 28235 to 54472
Data columns (total 15 columns):
countyid           41680 non-null category
latitude           41680 non-null int64
longitude          41680 non-null int64
yearbuilt          41680 non-null float64
bathroomcnt        41680 non-null float64
bedroomcnt         41680 non-null int64
house_area         41680 non-null float64
house_value        41680 non-null float64
land_value         41680 non-null float64
whole_area         41680 non-null float64
whole_value        41680 non-null float64
logerror           41680 non-null float64
transactiondate    41680 non-null datetime64[ns]
taxrate            41680 non-null float64
land_area          41680 non-null float64
dtypes: category(1), datetime64[ns](1), float64(10), int64(3)
memory usage: 6.1 MB
